<a href="https://colab.research.google.com/github/viswabhaskar1123/Epicsproject-2ndyear-/blob/main/selfexplain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
# from datasets import load_dataset
# Load BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [33]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
@file  : sst_dataset.py
@author: zijun
@contact : zijun_sun@shannonai.com
@date  : 2020/11/17 11:45
@version: 1.0
@desc  : sst5 and imdb task use the same dataset
"""
import os
from functools import partial

import torch
from transformers import RobertaTokenizer
from torch.utils.data import Dataset, DataLoader

# from datasets.collate_functions import collate_to_max_length


class SSTDataset(Dataset):

    def __init__(self, directory, prefix, bert_path, max_length: int = 512):
        super().__init__()
        self.max_length = max_length
        with open(os.path.join(directory, prefix + '.txt'), 'r', encoding='utf8') as f:
            lines = f.readlines()
        self.lines = lines
        self.tokenizer = AutoTokenizer.from_pretrained(bert_path)

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, idx):
        line = self.lines[idx]
        label, sentence = line.split('\t', 1)
        # delete .
        sentence = sentence.strip()
        if sentence.endswith("."):
            sentence = sentence[:-1]
        input_ids = self.tokenizer.encode(sentence, add_special_tokens=False)
        if len(input_ids) > self.max_length - 2:
            input_ids = input_ids[:self.max_length - 2]
        # convert list to tensor
        length = torch.LongTensor([len(input_ids) + 2])
        input_ids = torch.LongTensor([0] + input_ids + [2])
        label = torch.LongTensor([int(label)])
        return input_ids, label, length


def unit_test():
    root_path = "/content"
    bert_path = "bert-base-uncased"
    prefix = "train"
    dataset = SSTDataset(directory=root_path, prefix=prefix, bert_path=bert_path)

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=10,
        num_workers=0,
        shuffle=False,
        collate_fn=partial(collate_to_max_length, fill_values=[1, 0, 0])
    )
    for input_ids, label, length, start_index, end_index, span_mask in dataloader:
        print(input_ids.shape)
        print(start_index.shape)
        print(end_index.shape)
        print(span_mask.shape)
        print(label.view(-1).shape)
        print()


if __name__ == '__main__':
    unit_test()


Streaming output truncated to the last 5000 lines.
torch.Size([10])

torch.Size([10, 58])
torch.Size([1596])
torch.Size([1596])
torch.Size([10, 1596])
torch.Size([10])

torch.Size([10, 46])
torch.Size([990])
torch.Size([990])
torch.Size([10, 990])
torch.Size([10])

torch.Size([10, 37])
torch.Size([630])
torch.Size([630])
torch.Size([10, 630])
torch.Size([10])

torch.Size([10, 54])
torch.Size([1378])
torch.Size([1378])
torch.Size([10, 1378])
torch.Size([10])

torch.Size([10, 49])
torch.Size([1128])
torch.Size([1128])
torch.Size([10, 1128])
torch.Size([10])

torch.Size([10, 48])
torch.Size([1081])
torch.Size([1081])
torch.Size([10, 1081])
torch.Size([10])

torch.Size([10, 36])
torch.Size([595])
torch.Size([595])
torch.Size([10, 595])
torch.Size([10])

torch.Size([10, 37])
torch.Size([630])
torch.Size([630])
torch.Size([10, 630])
torch.Size([10])

torch.Size([10, 43])
torch.Size([861])
torch.Size([861])
torch.Size([10, 861])
torch.Size([10])

torch.Size([10, 47])
torch.Size([1035])
torch.

In [4]:
from transformers import BertConfig, BertModel


In [6]:
# encoding: utf-8
"""
@author: Yuxian Meng
@contact: yuxian_meng@shannonai.com

@version: 1.0
@file: radom_seed
@time: 2020/7/9 15:53
"""

import numpy as np
import torch


def set_random_seed(seed: int):
    """set seeds for reproducibility"""
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


if __name__ == '__main__':
    # without this line, x would be different in every execution.
    set_random_seed(0)

    x = np.random.random()
    print(x)


0.5488135039273248


In [7]:
# Import the SSTDataset class
# from datasets.sst_dataset import SSTDataset

# Define parameters for the dataset
directory = "/content/"
prefix = "train"  # Or any other prefix depending on your dataset
bert_path = "bert-base-uncased"
max_length = 512

# Instantiate the dataset
train_dataset = SSTDataset(directory=directory, prefix=prefix, bert_path=bert_path, max_length=max_length)


In [8]:

# Define parameters for the dataset
directory = "/content/"
prefix = "test"  # Or any other prefix depending on your dataset
bert_path = "bert-base-uncased"
max_length = 512

# Instantiate the dataset
val_dataset = SSTDataset(directory=directory, prefix=prefix, bert_path=bert_path, max_length=max_length)

In [ ]:
dataloader1 = DataLoader(
        dataset=train_dataset,
        batch_size=10,
        num_workers=0,
        shuffle=False,
        collate_fn=partial(collate_to_max_length, fill_values=[1, 0, 0])
    )
for input_ids, label, length, start_index, end_index, span_mask in dataloader1:
        print(input_ids)
        print(start_index)
        print(end_index)
        print(span_mask)
        print(label)
        print()
        break

tensor([[    0,  1996,  2600,  2003, 16036,  2000,  2022,  1996,  7398,  2301,
          1005,  1055,  2047,  1036,  1036, 16608,  1005,  1005,  1998,  2008,
          2002,  1005,  1055,  2183,  2000,  2191,  1037, 17624,  2130,  3618,
          2084,  7779, 29058,  8625, 13327,  1010,  3744,  1011, 18856, 19513,
          3158,  5477,  4168,  2030,  7112, 16562,  2140,     2,     1,     1,
             1,     1,     1,     1,     1],
        [    0,  1996,  9882,  2135,  9603, 13633,  1997,  1036,  1036,  1996,
          2935,  1997,  1996,  7635,  1005,  1005, 11544,  2003,  2061,  4121,
          2008,  1037,  5930,  1997,  2616,  2064,  2025, 23613,  6235,  2522,
          1011,  3213,  1032,  1013,  2472,  2848,  4027,  1005,  1055,  4423,
          4432,  1997,  1046,  1012,  1054,  1012,  1054,  1012, 23602,  1005,
          1055,  2690,  1011,  3011,     2],
        [    0,  3220,  1032,  1013,  4543,  8527,  5922, 16605,  1037, 22889,
          7974,  1997,  2774,  1011,  101

In [9]:
name='bert-base-uncased'
model=ExplainableModel(name)
# model1=ExplainableModelTrainer(model,train_dataset,val_dataset)

In [12]:
tokenizer =  AutoTokenizer.from_pretrained(name)

In [22]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [23]:
save_path = "/content/drive/MyDrive/my_experiments"


In [24]:
checkpoint_path = "/content/drive/MyDrive/checkpoints/model.pth"

In [ ]:
##

In [31]:
import torch
from torch import nn
from transformers import BertModel, BertConfig

class ExplainableModel(nn.Module):
    def __init__(self, bert_dir, num_labels=5):
        super().__init__()
        self.bert_config = BertConfig.from_pretrained(bert_dir)
        self.intermediate = BertModel.from_pretrained(bert_dir)
        self.span_info_collect = SICModel(self.bert_config.hidden_size)
        self.interpretation = InterpretationModel(self.bert_config.hidden_size)
        self.output = nn.Linear(self.bert_config.hidden_size, num_labels)

    def forward(self, input_ids, start_indexs, end_indexs, span_masks):
        attention_mask = (input_ids != 1).long()
        output = self.intermediate(input_ids, attention_mask=attention_mask)
        hidden_states = output.last_hidden_state
        h_ij = self.span_info_collect(hidden_states, start_indexs, end_indexs)
        H, a_ij = self.interpretation(h_ij, span_masks)
        out = self.output(H)
        return out, a_ij

class SICModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.W_1 = nn.Linear(hidden_size, hidden_size)
        self.W_2 = nn.Linear(hidden_size, hidden_size)
        self.W_3 = nn.Linear(hidden_size, hidden_size)
        self.W_4 = nn.Linear(hidden_size, hidden_size)

    def forward(self, hidden_states, start_indexs, end_indexs):
        W1_h = self.W_1(hidden_states)
        W2_h = self.W_2(hidden_states)
        W3_h = self.W_3(hidden_states)
        W4_h = self.W_4(hidden_states)

        W1_hi_emb = torch.index_select(W1_h, 1, start_indexs)
        W2_hj_emb = torch.index_select(W2_h, 1, end_indexs)
        W3_hi_start_emb = torch.index_select(W3_h, 1, start_indexs)
        W3_hi_end_emb = torch.index_select(W3_h, 1, end_indexs)
        W4_hj_start_emb = torch.index_select(W4_h, 1, start_indexs)
        W4_hj_end_emb = torch.index_select(W4_h, 1, end_indexs)

        span = W1_hi_emb + W2_hj_emb + (W3_hi_start_emb - W3_hi_end_emb) + torch.mul(W4_hj_start_emb, W4_hj_end_emb)
        h_ij = torch.tanh(span)
        return h_ij

class InterpretationModel(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.h_t = nn.Linear(hidden_size, 1)

    def forward(self, h_ij, span_masks):
        o_ij = self.h_t(h_ij).squeeze(-1)
        o_ij = o_ij - span_masks
        a_ij = nn.functional.softmax(o_ij, dim=1)
        H = (a_ij.unsqueeze(-1) * h_ij).sum(dim=1)
        return H, a_ij

def main():
    input_id_1 = torch.LongTensor([0, 4, 5, 6, 7, 2])
    input_id_2 = torch.LongTensor([0, 4, 5, 2])
    input_id_3 = torch.LongTensor([0, 4, 2])
    batch = [(input_id_1, torch.LongTensor([1]), torch.LongTensor([6])),
             (input_id_2, torch.LongTensor([1]), torch.LongTensor([4])),
             (input_id_3, torch.LongTensor([1]), torch.LongTensor([3]))]

    output = collate_to_max_length(batch=batch, fill_values=[1, 0, 0])
    input_ids, labels, length, start_indexs, end_indexs, span_masks = output

    assert isinstance(input_ids, torch.Tensor), "input_ids should be a tensor"

    bert_path = "bert-base-uncased"
    model = ExplainableModel(bert_path)
    print(model)

    output = model(input_ids=input_ids, start_indexs=start_indexs, end_indexs=end_indexs, span_masks=span_masks)
    print(output)

if __name__ == '__main__':
    main()


ExplainableModel(
  (intermediate): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [29]:
from typing import List
import numpy as np
import torch

def collate_to_max_length(batch: List[List[torch.Tensor]], max_len: int = None, fill_values: List[float] = None) -> List[torch.Tensor]:
    lengths = np.array([[len(field_data) for field_data in sample] for sample in batch])
    batch_size, num_fields = lengths.shape
    fill_values = fill_values or [0.0] * num_fields
    max_lengths = lengths.max(axis=0)
    if max_len:
        assert max_lengths.max() <= max_len
        max_lengths = np.ones_like(max_lengths) * max_len

    output = [torch.full([batch_size, max_lengths[field_idx]],
                         fill_value=fill_values[field_idx],
                         dtype=batch[0][field_idx].dtype)
              for field_idx in range(num_fields)]
    for sample_idx in range(batch_size):
        for field_idx in range(num_fields):
            data = batch[sample_idx][field_idx]
            output[field_idx][sample_idx][: data.shape[0]] = data

    max_sentence_length = max_lengths[0]
    start_indexs = []
    end_indexs = []
    for i in range(1, max_sentence_length - 1):
        for j in range(i, max_sentence_length - 1):
            start_indexs.append(i)
            end_indexs.append(j)

    span_masks = []
    for input_ids, label, length in batch:
        span_mask = []
        middle_index = input_ids.tolist().index(2)
        for start_index, end_index in zip(start_indexs, end_indexs):
            if 1 <= start_index <= length.item() - 2 and 1 <= end_index <= length.item() - 2 and (
                start_index > middle_index or end_index < middle_index):
                span_mask.append(0)
            else:
                span_mask.append(1e6)
        span_masks.append(span_mask)

    output.append(torch.LongTensor(start_indexs))
    output.append(torch.LongTensor(end_indexs))
    output.append(torch.LongTensor(span_masks))
    return output  # (input_ids, labels, length, start_indexs, end_indexs, span_masks)

def unit_test():
    input_id_1 = torch.LongTensor([0, 3, 2, 5, 6, 2])
    input_id_2 = torch.LongTensor([0, 3, 2, 4, 2])
    input_id_3 = torch.LongTensor([0, 3, 2])
    batch = [(input_id_1, torch.LongTensor([1]), torch.LongTensor([6])),
             (input_id_2, torch.LongTensor([1]), torch.LongTensor([5])),
             (input_id_3, torch.LongTensor([1]), torch.LongTensor([3]))]

    output = collate_to_max_length(batch=batch, fill_values=[1, 0, 0])
    print(output)

if __name__ == '__main__':
    unit_test()


[tensor([[0, 3, 2, 5, 6, 2],
        [0, 3, 2, 4, 2, 1],
        [0, 3, 2, 1, 1, 1]]), tensor([[1],
        [1],
        [1]]), tensor([[6],
        [5],
        [3]]), tensor([1, 1, 1, 1, 2, 2, 2, 3, 3, 4]), tensor([1, 2, 3, 4, 2, 3, 4, 3, 4, 4]), tensor([[      0, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,       0,
               0,       0],
        [      0, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,       0,
         1000000, 1000000],
        [      0, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000, 1000000,
         1000000, 1000000]])]


In [34]:
import json
import os
from functools import partial

import torch
from torch.nn import functional as F
from torch.nn.modules import CrossEntropyLoss
from torch.utils.data.dataloader import DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup, RobertaTokenizer

# from datasets.collate_functions import collate_to_max_length
# from datasets.sst_dataset import SSTDataset
# from datasets.snli_dataset import SNLIDataset
# from explain.model import ExplainableModel
# from utils.radom_seed import set_random_seed

set_random_seed(0)

class ExplainNLP(torch.nn.Module):

    def __init__(self, config):
        super(ExplainNLP, self).__init__()
        self.config = config
        self.bert_dir = config['bert_path']
        self.model = ExplainableModel(self.bert_dir)
        self.tokenizer = AutoTokenizer.from_pretrained(self.bert_dir)
        self.loss_fn = CrossEntropyLoss()

    def forward(self, input_ids, start_indexs, end_indexs, span_masks):
        return self.model(input_ids, start_indexs, end_indexs, span_masks)

    def compute_loss_and_acc(self, batch, mode='train'):
        input_ids, labels, length, start_indexs, end_indexs, span_masks = batch
        y = labels.view(-1)
        y_hat, a_ij = self(input_ids, start_indexs, end_indexs, span_masks)
        ce_loss = self.loss_fn(y_hat, y)
        reg_loss = self.config['lamb'] * a_ij.pow(2).sum(dim=1).mean()
        loss = ce_loss - reg_loss
        predict_scores = F.softmax(y_hat, dim=1)
        predict_labels = torch.argmax(predict_scores, dim=-1)
        acc = (predict_labels == y).float().mean()
        return loss, acc

    def get_dataloader(self, prefix="train"):
        dataset = SSTDataset(directory=self.config['data_dir'], prefix=prefix,
                             bert_path=self.bert_dir,
                             max_length=self.config['max_length'])
        dataloader = DataLoader(
            dataset=dataset,
            batch_size=self.config['batch_size'],
            num_workers=self.config['workers'],
            collate_fn=partial(collate_to_max_length, fill_values=[1, 0, 0]),
            drop_last=False
        )
        return dataloader

    def configure_optimizers(self):
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.config['weight_decay'],
            },
            {
                "params": [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters,
                          betas=(0.9, 0.98),
                          lr=self.config['lr'],
                          eps=self.config['adam_epsilon'])
        return optimizer

def train(config):
    if not os.path.exists(config['save_path']):
        os.mkdir(config['save_path'])

    model = ExplainNLP(config).to(config['device'])
    optimizer = model.configure_optimizers()

    train_dataloader = model.get_dataloader("train")
    val_dataloader = model.get_dataloader("dev")

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=config['warmup_steps'],
        num_training_steps=len(train_dataloader) * config['max_epochs']
    )

    best_val_acc = 0

    for epoch in range(config['max_epochs']):
        model.train()
        total_loss = 0
        total_acc = 0

        for batch in train_dataloader:
            batch = [item.to(config['device']) for item in batch]
            optimizer.zero_grad()
            loss, acc = model.compute_loss_and_acc(batch, mode='train')
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()
            total_acc += acc.item()

        avg_loss = total_loss / len(train_dataloader)
        avg_acc = total_acc / len(train_dataloader)

        print(f'Epoch {epoch + 1}/{config["max_epochs"]}, Train Loss: {avg_loss:.4f}, Train Acc: {avg_acc:.4f}')

        # Validation
        model.eval()
        with torch.no_grad():
            total_val_loss = 0
            total_val_acc = 0
            for batch in val_dataloader:
                batch = [item.to(config['device']) for item in batch]
                loss, acc = model.compute_loss_and_acc(batch, mode='dev')
                total_val_loss += loss.item()
                total_val_acc += acc.item()

            avg_val_loss = total_val_loss / len(val_dataloader)
            avg_val_acc = total_val_acc / len(val_dataloader)

            print(f'Epoch {epoch + 1}/{config["max_epochs"]}, Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}')

            if avg_val_acc > best_val_acc:
                best_val_acc = avg_val_acc
                torch.save(model.state_dict(), os.path.join(config['save_path'], 'best_model.pt'))

def evaluate(config):
    model = ExplainNLP(config).to(config['device'])
    model.load_state_dict(torch.load(config['checkpoint_path'], map_location=config['device']))
    test_dataloader = model.get_dataloader("test")

    model.eval()
    output = []
    check_data = []

    with torch.no_grad():
        total_test_loss = 0
        total_test_acc = 0
        for batch in test_dataloader:
            batch = [item.to(config['device']) for item in batch]
            loss, acc = model.compute_loss_and_acc(batch, mode='test')
            total_test_loss += loss.item()
            total_test_acc += acc.item()

        avg_test_loss = total_test_loss / len(test_dataloader)
        avg_test_acc = total_test_acc / len(test_dataloader)

        print(f'Test Loss: {avg_test_loss:.4f}, Test Acc: {avg_test_acc:.4f}')

    with open(os.path.join(config['save_path'], 'output.txt'), 'w', encoding='utf8') as f:
        f.writelines(output)
    with open(os.path.join(config['save_path'], 'test.txt'), 'w', encoding='utf8') as f:
        f.writelines(check_data)

def main():
    # Configuration settings
    config = {
        "bert_path": "bert-base-uncased",
        "batch_size": 10,
        "lr": 2e-5,
        "workers": 0,
        "weight_decay": 0.0,
        "adam_epsilon": 1e-9,
        "warmup_steps": 0,
        "max_length": 512,
        "data_dir": "/content/",
        "save_path": save_path,
        "save_topk": 5,
        "checkpoint_path": checkpoint_path,
        "span_topk": 5,
        "lamb": 1.0,
        "task": "sst5",
        "max_epochs": 10,
        "device": torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    }

    mode = "train"  # Change to "eval" for evaluation

    if mode == 'train':
        train(config)
    elif mode == 'eval':
        evaluate(config)
    else:
        raise Exception("unexpected mode!!!")

if __name__ == '__main__':
    main()


Epoch 1/10, Train Loss: 1.1125, Train Acc: 0.4968
Epoch 1/10, Val Loss: 1.6742, Val Acc: 0.3081


KeyboardInterrupt: 